<a href="https://colab.research.google.com/github/ncpitrsb/Aspect-based-Sentiment-Analysis/blob/main/Aspect_based_Sentiment_Analysis_with_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## SETUP

In [ ]:
!pip install tensorflow-gpu==2.3.1
!pip install ktrain

     |████████████████████████████████| 320.4MB 45kB/s 
     |████████████████████████████████| 460kB 37.8MB/s 
     |████████████████████████████████| 20.1MB 1.2MB/s 
ERROR: tensorflow 2.4.1 has requirement numpy~=1.19.2, but you'll have numpy 1.18.5 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement tensorflow-estimator<2.5.0,>=2.4.0, but you'll have tensorflow-estimator 2.3.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5


     |████████████████████████████████| 25.3MB 63.8MB/s 
     |████████████████████████████████| 6.8MB 48.8MB/s 
     |████████████████████████████████| 983kB 38.1MB/s 
     |████████████████████████████████| 266kB 39.0MB/s 
     |████████████████████████████████| 1.9MB 42.3MB/s 
     |████████████████████████████████| 1.2MB 22.6MB/s 
     |████████████████████████████████| 471kB 30.6MB/s 
     |████████████████████████████████| 890kB 44.2MB/s 
     |████████████████████████████████| 3.2MB 38.2MB/s 
  Created wheel for ktrain: filename=ktrain-0.26.1-cp37-none-any.whl size=25277418 sha256=5e3e53a213b8992b5e6693367f6122298ffd47110192f964574f9a5e5715e675
  Stored in directory: /root/.cache/pip/wheels/5f/e6/26/e587644c4f5bd1a32d24d7bda0b132f8f2d4dfbe0bca62fe70
  Created wheel for langdetect: filename=langdetect-1.0.8-cp37-none-any.whl size=993193 sha256=56f47a123bc13548ff8d9ea064b073b0c2adc8bb0150b63aa1b3951d9fb71ee1
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d

In [ ]:
!gdown --id 1s-8A8sF7b23Tb9Myoc_3DTl6YXLpL17L

Downloading...
From: https://drive.google.com/uc?id=1s-8A8sF7b23Tb9Myoc_3DTl6YXLpL17L
To: /content/lab4_train.csv
100% 331k/331k [00:00<00:00, 44.6MB/s]


In [ ]:
!gdown --id 1YtAHCzeZUXGZQ9cimdkkUq4lUk3ZH-I_

Downloading...
From: https://drive.google.com/uc?id=1YtAHCzeZUXGZQ9cimdkkUq4lUk3ZH-I_
To: /content/evaluate.py
100% 7.03k/7.03k [00:00<00:00, 8.97MB/s]


In [ ]:
import ktrain
from ktrain import text
import pandas as pd
from tqdm import tqdm
import pickle
import numpy as np

## **Data preprocessing**



In [ ]:
def StratifiedSplit(dat,label_name,split_ratio):
  total_row=dat.count()[0]
  labels=dat[label_name].unique()
  ratio_dict={}
  for i in labels:
    ratio_dict[i]=dat.where(dat[label_name]==i).dropna().count()[0]/total_row
  num_sample=int(total_row*split_ratio)
  test_df=pd.DataFrame(columns=dat.columns)
  train_df=pd.DataFrame(columns=dat.columns)
  for i in labels:
    num_sample_label =int(num_sample*ratio_dict[i])
    dat_sm=dat.where(dat[label_name]==i).dropna()
    dat_sm=dat_sm.sample(frac=1)
    msk = np.random.rand(dat_sm.count()[0]) < (num_sample_label/dat_sm.count()[0])
    test_df=test_df.append(dat_sm[msk])
    train_df=train_df.append(dat_sm[~msk])
  return (train_df, test_df)

In [ ]:
df = pd.read_csv('/content/lab4_train.csv')

In [ ]:
df.polarity.value_counts().plot.bar()

In [ ]:
df.aspectCategory.value_counts().plot.bar()

In [ ]:
df["polarity"] = df["polarity"].replace({'neutral': 0, 'negative': 1, 'positive':2,'conflict':3})

In [ ]:
train, test=StratifiedSplit(df,"polarity",0.2)
x_train = train["text"].values.tolist()
y_train =  train["polarity"].values.tolist()
x_test = test["text"].values.tolist()
y_test = test["polarity"].values.tolist()

In [ ]:
(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=x_train, y_train=y_train,
                                                                       x_test=x_test, y_test=y_test,
                                                                       class_names=['0','1','2','3'],
                                                                       preprocess_mode='bert',
                                                                       ngram_range=1, 
                                                                       maxlen=350)

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


/usr/local/lib/python3.7/dist-packages/ktrain/utils.py:605: UserWarning: class_names implies classification but targets array contains float(s) instead of integers or strings
  warnings.warn('class_names implies classification but targets array contains float(s) instead of integers or strings')


task: text classification


## Model Training

In [ ]:
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)
learner = ktrain.get_learner(model, train_data=(x_train, y_train), batch_size=6)
model.summary()

Is Multi-Label? False
maxlen is 350
done.
Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 350)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 350)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 350, 768), ( 23440896    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 350, 768)     1536        Input-Segment[0][0]              
_____________________________________________

In [ ]:
hist = learner.fit_onecycle(2e-5, 2) 



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/2
416/416 [==============================] - 10136s 24s/step - loss: 0.9256 - accuracy: 0.6673
Epoch 2/2
416/416 [==============================] - 10165s 24s/step - loss: 0.5523 - accuracy: 0.7945


In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)

Predict to compare Aj

In [ ]:
df_train = pd.read_csv('/content/lab4_train.csv')
text_train = df_train['text'].values.tolist()

In [ ]:
pred_y_train = []
for i in tqdm(range(len(text_train))):
    pred_y_train.append(predictor.predict(text_train[i]))

Predict Test

In [ ]:
df_test = pd.read_csv('/content/lab4_test.csv')
text_test = df_test['text'].values.tolist()

In [ ]:
pred_y_test = []
for i in tqdm(range(len(text_test))):
    pred_y_test.append(predictor.predict(text_test[i]))

Model Saving

In [ ]:
model.save('sentiment_model.h5')  # creates a HDF5 file 'my_model.h5'
del model  # deletes the existing model


In [ ]:
model.save_weights('sentiment_model_weights.h5')

## Evaluate

Evaluate Train Dataset 80% (2729 samples)

In [ ]:
eva_train = df_train.drop(columns=['polarity'])
eva_train['polarity'] = pred_y_train

eva_train["polarity"] = eva_train["polarity"].replace({'0':'neutral',
                                                       '1':'negative',
                                                       '2':'positive',
                                                       '3':'conflict'})

eva_train.to_csv('eva_train.csv',index=False)

In [ ]:
!python evaluate.py lab4_train.csv .csv

INCORRECT COLUMN NAME : must have "id", "aspectCategory", "polarity"


In [ ]:
peace = pd.read_csv('ee.csv')

In [ ]:
peace

,aspectCategory,polarity
0,service,negative
1,food,positive
2,food,negative
3,food,positive
4,service,positive
...,...,...
3151,anecdotes/miscellaneous,positive
3152,service,positive
3153,anecdotes/miscellaneous,positive
3154,food,positive


In [ ]:
earth = pd.read_csv('/content/eva_train.csv')

In [ ]:
earth.drop(columns=['text'],inplace=True)
earth['aspectCategory'] = peace['aspectCategory']

In [ ]:
earth.to_csv('earth.csv',index=False)

In [ ]:
!python evaluate.py lab4_train.csv earth.csv

=== CLASSIFICATION : ASPECT ===
                class name  precision  recall  F1-score support
0                     food      0.965   0.770     0.857    1051
1                    price      0.950   0.695     0.803     275
2                  service      0.962   0.747     0.841     506
3                 ambience      0.948   0.644     0.767     368
4  anecdotes/miscellaneous      0.939   0.888     0.913     954
5                MACRO AVG      0.953   0.749     0.836    3154
6                MICRO AVG      0.953   0.781     0.858    3154 

=== CLASSIFICATION : SENTIMENT ===
  class name  precision  recall  F1-score support
0   positive      0.941   0.932     0.936    1546
1   negative      0.853   0.835     0.844     635
2    neutral      0.821   0.698     0.754     387
3   conflict      0.696   0.441     0.540     161
4  MACRO AVG      0.828   0.726     0.769    2729
5  MICRO AVG      0.895   0.847     0.870    2729 

=== CLASSIFICATION : OVERALL ===
              precision  recall  F

Evaluate Test Dataset (461 samples)

In [ ]:
eva_test = df_test.drop(columns=['text'])
eva_test['polarity'] = pred_y_test
eva_test["polarity"] = eva_test["polarity"].replace({'0':'neutral', '1':'negative', '2':'positive','3':'conflict'})
eva_test.to_csv('eva_test.csv',index=False)